In [ ]:
#default_exp parser

# Parser_evaluator
> Blah blah

In this notebook we'll develop a formula parser for Excel formulas. The goal is to take a string as input and produce an AST. The goal is to produce a parser that can handle cell references, functions, and the basic operators.

Here's the top-level Excel spec:

`formula=expression  ;
expression="(",  expression,  ")"  | constant  | prefix-operator,  expression  | expression,  infix-operator,  expression  | expression,  postfix-operator  | cell-reference  |function-call  | name  ;`

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import ebb.types as t

In [ ]:
#export
def parse(s):
    '''Parse s into an AST.'''
    assert isinstance(s, str), f'Argument {s} to parse is not a string'
    # formula such as a reference
    if s.startswith('='): return parse_formula(s[1:].strip())
    else: return parse_value(s)
    
def parse_value(s):
    # text
    # Todo(Rik): this doesn't work for formulas with tuples in them. Should be not greedy!
    if len(s) >= 2 and s.startswith('"') and s.endswith('"') and not '"' in s[1:-1]: return s[1:-1]
    # Single char
    elif not s in '0123456789' and len(s) == 1: return s
    # Bools
    elif s.lower() == 'true': return True
    elif s.lower() == 'false': return False
    # Integers
    try: return int(s)
    except ValueError: pass
    # Floats
    try: return float(s)
    except ValueError: pass
    raise t.ParseError(f'Unable to parse value {s}')

In [ ]:
#export
import re

def get_parenthesized_indices(s):
    '''Get a set of indices in s of characters which occur on or between parentheses'''
    paren_indices = set()
    n_spans_open = 0
    for i, c in enumerate(s):
        if c == '(': n_spans_open +=1 
        
        if n_spans_open > 0: paren_indices.add(i)

        if c == ')': n_spans_open -= 1
        if n_spans_open < 0: raise t.ParseError(f'Unmatched ) in {s}')
        
    if n_spans_open > 0: raise t.ParseError(f'Unmatched ( in {s}')
    return paren_indices

assert get_parenthesized_indices('(2+3)*(4*(7-3))') == set(range(0, 5)) | set(range(6, 15))
assert get_parenthesized_indices('fooo (3, 4) bar ((5, (7)))') == set(range(5, 11)) | set(range(16, 26))


def parse_formula(s):
    '''Turn a formula (after some = sign) into an AST'''
    # Base case, somebody put a constant there
    try: return parse(s)
    except t.ParseError: pass
    # Parse ref
    try: return t.Ref.from_string(s)
    except t.ParseError: pass
        
    # Matching operators: we do this in __reverse__ precendence order. The intuition for this is that
    # during calculation, we roll up the parse tree from the bottom, since that's where the leaves with
    # values are. Since the strongest binding operations hould be executed first, it follows that we
    # want to push those operators down into the 
    # Operators are in precedence order, so start by identifying the *last* thing that should match.
    # We try to parse the sections identified by the parts indicated by the formula. If that fails, clearly
    # we must have misinterpreted the operation (like for -3.0, we'll first try to parse it as
    # InfixOp('-', '', '3.0')) but '' doesn't parse (text values we want to have quotes).
    indices_to_skip = get_parenthesized_indices(s)
    for op, typ in reversed(t.operators.items()):
        # Todo(Rik): special-casing for space operator. It's only valid between two references,
        # should just be stripped otherwise. Bit of a hassle.
        # Todo(Rik): did not think through tuples well enough: this is a case where a higher-priority
        # operator can follow a lower priority one, e.g. in '=IF(3<4, 10, 11)'. Maybe this is a precendence
        # error, and tuple should be somewhere else in the hierarchy?
        # Todo(Rik): similary, tuples might get empty arguments [(1,,1) should evaluate to (1, None, 1)]
        # but this is obviously nonsensical for the others.
        op_whitespace = r'\s*'+op+r'\s*'
        if re.search(op_whitespace, s) and typ == t.InfixOp:
            for m in re.finditer(op_whitespace, s):  # Sure hope nothing's left associative
                if m.start() in indices_to_skip: continue
                left, right = s[:m.start()], s[m.end():]
                try: return typ(op, parse_formula(left), parse_formula(right))
                except t.ParseError: pass
        elif re.search(op_whitespace, s) and typ == t.PostfixOp:
            # If parsing was correct, should have been the last one
            if s.index(op) in indices_to_skip: continue
            if s.index(op) != len(s)-1: raise ParseError(f'PostfixOp {op} not in last position in {s}')
            try: return typ(op, parse_formula(s[:-1]))
            except t.ParseError: pass
        elif re.search(op_whitespace, s) and typ == t.PrefixOp:
            # Everything following op should be parseable as one expression
            if s.index(op) in indices_to_skip: continue
            if s.index(op) != 0: raise ParseError(f'PrefixOp {op} not in first position in {s}')
            try: return typ(op, parse_formula(s[1:]))
            except t.ParseError: pass
          
    # There are no operators outside of parentheses to parse. that means that we must've arrived
    # at an enclosing expression. Either something like '(3+4)', or something like 'SUM(A1:A4)'.
    # We use `find` to distinguish between the cases:
    if s == '' or s[-1] != ')': raise ParseError(f'{s} does not appear to be a parseable formula.')
    i_open = s.find('(')
    if i_open == 0: return parse_formula(s[1:-1])
    elif i_open > 0:
        if s[i_open+1:-1] == '': return Function(name=s[:i_open], args=None)
        else: return t.Function(name=s[:i_open], args=parse_formula(s[i_open+1:-1]))
        
    raise t.ParseError(f'{s} does not appear to be a parseable formula.')

In [ ]:
# Individual values
assert parse('-25') == -25
assert parse('10.3') == 10.3
assert parse('TRUE') == True
assert parse('FALSE') == False
assert parse('\"foo\"') == 'foo'
assert parse("1.3e-7") == 1.3e-7
assert parse("-1.3e6") == -1.3e6
assert parse('c') == 'c'  # We allow single characters
assert parse('1e7') == 1e7

In [ ]:
import pytest
# Easy formulas
assert parse('=3.0') == 3.0
assert parse('=B7') == t.Ref(6, 1, fixed_row=False, fixed_column=False)
assert parse('=$B$7') == t.Ref(6, 1, fixed_row=True, fixed_column=True)
assert parse('=3*4') == t.InfixOp(op=r'\*', left=3, right=4)
assert parse('=2+3*4') == t.InfixOp(op=r'\+', left=2, right=t.InfixOp(op=r'\*', left=3, right=4))
assert parse('=3^4') == t.InfixOp(op=r'\^', left=3, right=4)
with pytest.raises(t.ParseError):
    parse('=foo-')
    
# Don't forget spacing issues!
assert parse('= 3.0') == 3.0

In [ ]:
assert parse('=(2+3)*4') == t.InfixOp(op=r'\*', left=t.InfixOp(op=r'\+', left=2, right=3), right=4)
assert parse('=2+3*4') == t.InfixOp(op=r'\+', left=2, right=t.InfixOp(op=r'\*', left=3, right=4))
assert parse('=SUM(A1:A4)') == t.Function(
    name='SUM',
    args=t.InfixOp(op=':', left=t.Ref(row=0, column=0), right=t.Ref(row=3, column=0))
)
assert parse('=PI()') == t.Function(name='PI', args=None)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted evaluator.ipynb.
Converted parser.ipynb.
Converted prototype.ipynb.
Converted translator.ipynb.
Converted types.ipynb.
Converted util.ipynb.
